In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [10]:
data = np.loadtxt('../data/data_zero_filter_03_50.csv', delimiter = ',', skiprows = 1)
data.shape

(22199, 2403)

In [36]:
data_file = '../data/info.csv'
df = pd.read_csv(data_file)
df = df.loc[:,['id','nst_result']]
#label[label['nst_result']==3].loc['nst_result'] = 2
#label.where(label['nst_result'] == 3).count()
# label_1 = df.where(df['nst_result']==1)
# label_2 = df.where(df['nst_result']==2)
# label_3 = df.where(df['nst_result']==3)
# label_1.shape

(32184, 2)

In [14]:
data_file= '../data/data_zero_filter_03_50.csv'
label_file = '../data/info.csv'
data = pd.read_csv(data_file)
df = pd.read_csv(label_file)
label = df.loc[:,['id','nst_result']]
data_label = pd.merge(data, label, how = 'left', left_index = True, left_on = 'id', right_on = 'id')

#data_label.where(data_label['nst_result'] == 3).count()

In [15]:
data_label.loc[data_label['nst_result'] == 3, 'nst_result'] = 2
data_label.loc[data_label['nst_result'] == 4, 'nst_result'] = np.nan
data_label.dropna()

#data_label.where(data_label['nst_result'] == 4).count()
#del data_label.iloc[nst_4[id]]

,id,y0,y1,y2,y3,y4,y5,y6,y7,y8,...,y2393,y2394,y2395,y2396,y2397,y2398,y2399,y2400,y2401,nst_result
0,726,134.0,134.0,133.0,133.0,133.0,133.0,133.0,133.0,135.0,...,135.0,134.0,134.0,127.0,127.0,126.0,127.0,129.0,130.0,2.0
1,727,130.0,126.0,126.0,126.0,125.0,123.0,122.0,122.0,121.0,...,128.0,129.0,129.0,127.0,127.0,124.0,124.0,123.0,123.0,2.0
2,728,147.0,150.0,150.0,155.0,155.0,158.0,158.0,158.0,158.0,...,161.0,161.0,160.0,160.0,158.0,158.0,154.0,154.0,151.0,1.0
3,729,83.0,83.0,83.0,83.0,83.0,83.0,166.0,166.0,159.0,...,180.0,182.0,182.0,177.0,176.0,176.0,182.0,180.0,178.0,1.0
4,732,142.0,143.0,143.0,141.0,141.0,141.0,141.0,141.0,141.0,...,153.0,153.0,154.0,154.0,153.0,155.0,155.0,155.0,155.0,2.0
5,733,141.0,138.0,138.0,138.0,138.0,141.0,141.0,140.0,140.0,...,89.0,89.0,89.0,89.0,89.0,124.0,124.0,127.0,128.0,2.0
6,735,65.5,131.0,131.0,135.0,135.0,135.0,135.0,134.0,134.0,...,146.0,146.0,155.0,155.0,155.0,155.0,153.0,153.0,151.0,2.0
7,742,115.0,115.0,115.0,115.0,115.0,115.0,119.0,119.0,111.0,...,116.0,117.0,118.0,118.0,118.0,120.0,120.0,125.0,125.0,2.0
8,748,119.0,119.0,120.0,120.0,120.0,120.0,120.0,120.0,119.0,...,133.0,132.0,133.0,132.0,132.0,132.0,129.0,128.0,129.0,2.0
10,753,143.0,139.0,139.0,139.0,139.0,139.0,139.0,139.0,139.0,...,152.0,154.0,154.0,154.0,154.0,155.0,155.0,155.0,155.0,1.0


In [107]:
    data_file = '../data/data_zero_filer_03_50.csv'
    label_file = '../data/info.csv'
    data = pd.read_csv(data_file)
    print(data.shape)
    df = pd.read_csv(label_file)
    label = df.loc[:, ['id', 'nst_result']]
    data_label = pd.merge(data, label, how='left', left_index=True, left_on='id', right_on='id')
    # 把'异常型' 归入 '可疑型'
    data_label.loc[data_label['nst_result'] == 3, 'nst_result'] = 2
    # 剔除'无法判读'型
    data_label.loc[data_label['nst_result'] == 4, 'nst_result'] = np.nan
    data_label.dropna(inplace = True)
    print(data_label.shape)

(22199, 2403)
(22193, 2404)


In [112]:
data_label.drop('id', axis = 1, inplace = True)
data_label

ValueError: labels ['id'] not contained in axis

In [2]:
df = pd.DataFrame(np.arange(16).reshape(4,4),
                      columns=['A', 'B', 'C', 'D'])
df['label'] = pd.DataFrame(np.arange(4))
df

,A,B,C,D,label
0,0,1,2,3,0
1,4,5,6,7,1
2,8,9,10,11,2
3,12,13,14,15,3


In [3]:
data = pd.read_csv('../data/zero_count.csv')
data['id'] = pd.DataFrame(np.arange(24360)) 
data

,sample_id,zero_count,id
0,334622,2401,0
1,15592,2283,1
2,22380,2265,2
3,22432,2255,3
4,15739,2175,4
5,2108842,1940,5
6,355407,1717,6
7,14568,1707,7
8,612884,1653,8
9,1129714,1591,9


In [4]:
pd.crosstab(data.id,data.zero_count)
plt.title('zero count in original data')
plt.xlabel('samples')
plt.ylabel('number of zero_value')
plt.savefig('zero_count_in_original_data')